In [1]:
#Hyperparameters:

SHOULD_USE_SEED = True
RADIUS = 10000
BATCH_SIZE = 700
EPSILON = 0.1
SIGMA = 0.0001
MAX_ITERS = 100

TRAIN_FILE = "snli_1.0_train.txt"
VAL_FILE = "snli_1.0_dev.txt"
TEST_FILE = "snli_1.0_test.txt"


In [2]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import numpy as np

from data.data_creator import data_create_SNLI

if SHOULD_USE_SEED:
    np.random.seed(42)


## Dataset 

In [3]:
data, labels = data_create_SNLI()

In [4]:
#Splitting the dataset by using the original split

x_train, x_val, x_test = data[TRAIN_FILE], data[VAL_FILE], data[TEST_FILE]
y_train, y_val, y_test = labels[TRAIN_FILE], labels[VAL_FILE], labels[TEST_FILE]

## Black-box model - Multinomial Naive Bayes classifier

The black box model that was used in the original article.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

data_multinomial_nb = dict()

for key in data:
    data_multinomial_nb[key] = \
        [first + ', ' + second for first, second in zip(data[key]['premise'], data[key]['hypothesis'])]
    
x_train, x_val, x_test = data_multinomial_nb[TRAIN_FILE], data_multinomial_nb[VAL_FILE], data_multinomial_nb[TEST_FILE]

vect_text = TfidfVectorizer(use_idf = False)
x_vec_train = vect_text.fit_transform(x_train)

clf = MultinomialNB().fit(x_vec_train, y_train)

In [6]:
preds = clf.predict(vect_text.transform(x_val))

In [7]:
print('Val accuracy', metrics.accuracy_score(y_val, preds))

Val accuracy 0.5363


## Black-box model - Multinomial Naive Bayes classifier

The black box model above doesn't give a good result. Therefore, we use another black box model presented in the following article:
https://nlp.stanford.edu/pubs/snli_paper.pdf

### Data preperation


In [10]:
from torchtext.legacy.data import Field

x_train, x_val, x_test = data[TRAIN_FILE], data[VAL_FILE], data[TEST_FILE] 

text_field = Field(
    tokenize='basic_english', 
    lower=True
)
label_field = Field(sequential=False, use_vocab=False)
# sadly have to apply preprocess manually

preprocessed_text = [text_field.preprocess(x) for x in x_train['hypothesis']]#.apply(lambda x: text_field.preprocess(x))
# load fastext simple embedding with 300d
text_field.build_vocab(
     preprocessed_text
)
vocab = text_field.vocab



In [11]:
print(x_train['premise'][0])

A person on a horse jumps over a broken down airplane.


## Instance to explain

In [ ]:
x_explain = x_test[1]#"the movie's thesis -- elegant technology for the masses -- is surprisingly refreshing ."
print('x to explain: ',x_explain)
print('Predicted class: ', clf.predict(vect_text.transform([x_explain]))[0])
print('True class: ', y_test[1])
print('Predict probablilities: ', clf.predict_proba(vect_text.transform([x_explain]))[0])


## Building MeLime model:

In [ ]:
import nltk
from torch.utils.data import DataLoader
def tokenizer(x):
    return x.split()
dl_train = [tokenizer(x) for x in x_train]

In [ ]:
from gen_models.word2vec_gen import Word2VecGen, Word2VecEncoder
#The radius is <radius> most similar words
generator = Word2VecGen(encoder = Word2VecEncoder(dl_train), corpus = x_train, radius = RADIUS, tokenizer = tokenizer)

In [ ]:
from interpretable_local_models.statistics_model import StatisticsLocalModel
tokenized_x_explain = x_explain.split()
y_p_explain = max(clf.predict_proba(vect_text.transform([x_explain]))[0])
explainer_model = StatisticsLocalModel(y_p_explain, len(tokenized_x_explain), tokenizer)
print(tokenized_x_explain)

In [ ]:
from MeLime.model import MeLimeModel

def transform_func(x):
    return vect_text.transform([x])

model = MeLimeModel(black_box_model = clf,gen_model =generator, batch_size = BATCH_SIZE, epsilon_c = EPSILON, 
                    sigma = SIGMA, explainer_model = explainer_model, transform_func = transform_func, 
                    max_iters = MAX_ITERS, tokenizer = tokenizer)

## Explaining the instance

In [ ]:
res, sentences_with_probs = model.forward(x_explain)

## Plotting results

In [ ]:
ax = StatisticsLocalModel.plot_explaination(res)


In [ ]:
import seaborn as sns
StatisticsLocalModel.plot_sentence_heatmap(res)

## Plotting most favorable and contrary samples phrases:

Favorable sentence - a generated sentence using Word2VecGen that improves the model's confidence in its 
prediction on the original sentence.

Contrary samples - a generated sentence using Word2VecGen that decrease the model's confidence in its prediction on the original sentence and <b>might even change its prediction on the generated sentence</b>.

### Most contrary samples phrases:



In [ ]:
sorted(sentences_with_probs, key = lambda x: x[1])[:5]

### Most favorable samples phrases:

In [ ]:
sorted(sentences_with_probs, key = lambda x: x[1], reverse = True)[:5]